In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D, BatchNormalization, Activation, Add, SeparableConv2D, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# Verify GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

base_dir = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\data_aug_3'
subfolders = ['clear', 'clouds']
categories = ['Healthy_augmented', 'Damaged_augmented']
IMG_HEIGHT =  32
IMG_WIDTH = 32
BATCH_SIZE = 32
EPOCHS = 100  # Set to 100

# Load Data Function
def load_data(base_dir, subfolders, categories, img_height, img_width):
    data = []
    labels = []
    image_paths = []
    for category in categories:
        class_num = categories.index(category)
        for subfolder in subfolders:
            folder_path = os.path.join(base_dir, subfolder, category)
            images = sorted(os.listdir(folder_path))
            for img_name in images:
                if img_name.endswith('.png'):
                    img_path = os.path.join(folder_path, img_name)
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    data.append(img_array)
                    labels.append(class_num)
                    image_paths.append((subfolder, category, img_name))
    return np.array(data), np.array(labels), image_paths

# Load the data
data, labels, image_paths = load_data(base_dir, subfolders, categories, IMG_HEIGHT, IMG_WIDTH)
data = data / 255.0

# Split the data
def split_data(image_paths):
    unique_image_ids = list(set([img_name for subfolder, category, img_name in image_paths]))
    train_ids, test_ids = train_test_split(unique_image_ids, test_size=0.2, random_state=42)
    train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2
    return train_ids, val_ids, test_ids

def get_split_indices(image_paths, split_ids):
    split_indices = [i for i, (subfolder, category, img_name) in enumerate(image_paths) if img_name in split_ids]
    return split_indices

train_ids, val_ids, test_ids = split_data(image_paths)
train_indices = get_split_indices(image_paths, train_ids)
val_indices = get_split_indices(image_paths, val_ids)
test_indices = get_split_indices(image_paths, test_ids)

# Prepare train, validation, and test sets
X_train, y_train = data[train_indices], labels[train_indices]
X_val, y_val = data[val_indices], labels[val_indices]
X_test, y_test = data[test_indices], labels[test_indices]

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

# Data augmentation
def create_datagen(seed=None):
    return ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    ), seed

datagen, seed = create_datagen(seed=42)
datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, seed=seed)
val_generator = datagen.flow(X_val, y_val, batch_size=BATCH_SIZE, seed=seed)

# Compute class weights using the training set
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

# Best hyperparameters from Optuna (Provided by You)
best_model_params = {
    'num_residual_blocks': 4, 
    'filters': 121, 
    'kernel_size': 5, 
    'dense_units': 1387, 
    'dropout_rate': 0.40846071442232423, 
    'learning_rate': 0.0002469909302048313
}

# Define ablation configurations
ABLATION_CONFIGS = {
    "baseline": {"use_residual": True, "use_attention": True, "use_dropout": True, "use_batch_norm": True},
    "no_residual": {"use_residual": False, "use_attention": True, "use_dropout": True, "use_batch_norm": True},
    "no_attention": {"use_residual": True, "use_attention": False, "use_dropout": True, "use_batch_norm": True},
    "no_dropout": {"use_residual": True, "use_attention": True, "use_dropout": False, "use_batch_norm": True},
    "no_batch_norm": {"use_residual": True, "use_attention": True, "use_dropout": True, "use_batch_norm": False},
}

# Function to build model with ablation configuration
def build_model(config):
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    
    # First convolutional layer with the best filter size and kernel size
    x = Conv2D(best_model_params['filters'], (best_model_params['kernel_size'], best_model_params['kernel_size']), padding='same', activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)

    # Add residual blocks based on the best hyperparameters
    for _ in range(best_model_params['num_residual_blocks']):
        if config["use_residual"]:
            x = residual_block(x, best_model_params['filters'], best_model_params['kernel_size'], config)
        if config["use_attention"]:
            x = attention_block(x, best_model_params['filters'])
        if x.shape[1] >= 2 and x.shape[2] >= 2:
            x = MaxPooling2D((2, 2))(x)
    
    x = GlobalAveragePooling2D()(x)

    # Dropout based on the best dropout rate
    if config["use_dropout"]:
        x = Dropout(best_model_params['dropout_rate'])(x)
    
    # Dense layer using the best number of units
    x = Dense(best_model_params['dense_units'], activation='relu')(x)
    outputs = Dense(2, activation='softmax')(x)

    # Compile the model with the best learning rate
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=best_model_params['learning_rate']), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Residual block definition
def residual_block(x, filters, kernel_size, config):
    shortcut = x
    x = SeparableConv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    if config["use_batch_norm"]:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = SeparableConv2D(filters, (kernel_size, kernel_size), padding='same')(x)
    if config["use_batch_norm"]:
        x = BatchNormalization()(x)
    
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), padding='same')(shortcut)
        if config["use_batch_norm"]:
            shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# Attention block definition
def attention_block(x, filters):
    attention = MultiHeadAttention(num_heads=8, key_dim=filters)(x, x)
    attention = Add()([x, attention])
    return attention

# Function to train and evaluate model for ablation study
def train_and_evaluate_model(config_name, config, X_train, y_train, X_val, y_val, class_weights):
    print(f"Training model with {config_name} configuration...")
    model = build_model(config)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)
    
    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    val_accuracy = np.max(history.history['val_accuracy'])
    return model, val_accuracy

# Perform ablation study
def ablation_study(X_train, y_train, X_val, y_val, X_test, y_test, class_weights):
    results = {}
    for config_name, config in ABLATION_CONFIGS.items():
        model, val_accuracy = train_and_evaluate_model(config_name, config, X_train, y_train, X_val, y_val, class_weights)
        
        # Evaluate on test set
        test_predictions = model.predict(X_test)
        y_test_true = np.argmax(y_test, axis=1)
        y_test_pred = np.argmax(test_predictions, axis=1)
        test_conf_matrix = confusion_matrix(y_test_true, y_test_pred)
        test_class_report = classification_report(y_test_true, y_test_pred, target_names=categories)
        
        # Store results
        results[config_name] = {
            "val_accuracy": val_accuracy,
            "confusion_matrix": test_conf_matrix,
            "classification_report": test_class_report
        }
        
        print(f"Validation Accuracy for {config_name}: {val_accuracy}")
        print(f"Confusion Matrix (Test) for {config_name}:")
        print(test_conf_matrix)
        print(f"Classification Report (Test) for {config_name}:")
        print(test_class_report)

    return results

# Run the ablation study
ablation_results = ablation_study(X_train, y_train, X_val, y_val, X_test, y_test, class_weights)

# Output the results
for config_name, result in ablation_results.items():
    print(f"Results for {config_name}:")
    print(f"Validation Accuracy: {result['val_accuracy']}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}")
    print(f"Classification Report:\n{result['classification_report']}")


TensorFlow version: 2.10.0
Num GPUs Available:  1
Training model with baseline configuration...
Epoch 1/100
226/226 [==============================] - 65s 244ms/step - loss: 0.7676 - accuracy: 0.5089 - val_loss: 0.6931 - val_accuracy: 0.5153 - lr: 2.4699e-04
Epoch 2/100
226/226 [==============================] - 54s 238ms/step - loss: 0.7245 - accuracy: 0.5021 - val_loss: 0.6975 - val_accuracy: 0.4205 - lr: 2.4699e-04
Epoch 3/100
226/226 [==============================] - 54s 239ms/step - loss: 0.7025 - accuracy: 0.5202 - val_loss: 0.6880 - val_accuracy: 0.5484 - lr: 2.4699e-04
Epoch 4/100
226/226 [==============================] - 54s 238ms/step - loss: 0.6911 - accuracy: 0.5353 - val_loss: 0.6750 - val_accuracy: 0.5861 - lr: 2.4699e-04
Epoch 5/100
226/226 [==============================] - 54s 239ms/step - loss: 0.6851 - accuracy: 0.5701 - val_loss: 0.6859 - val_accuracy: 0.5882 - lr: 2.4699e-04
Epoch 6/100
226/226 [==============================] - 54s 239ms/step - loss: 0.6811 - ac


KeyboardInterrupt

